# Notebook 3: **SCRAPING** 

Attraverso la libreria BeautifulSoup, si vogliono scaricare i dati dalla pagina di "www.rogertebert.com" . 

La **prima iterazione** viene fatta sul numero di pagine: sono stati scaricati i nomi di tutti i film con data di rilascio dal 2010, disponibili sul sito web, insieme ai corrispondenti link che portano alla loro recensione completa. In questo modo, per ogni pagina sono stati ottenuti 28 film, di cui gli ultimi 4 apparivano essere sempre gli stessi, ovvero “Top Gun: Maverick”, “Interceptor”, “RRR”, “Eiffel”. Questi ultimi infatti, appartenevano alla categoria di film con “Popular Reviews”. Sono stati rimossi i duplicati.<br> 
La **seconda iterazione** viene fatta sui link ottenuti nel punto precedente: vengono scaricate tutte le recensioni. Esse sono composte da diversi paragrafi, per cui è stato necessario unire i *tag* relativi attraverso un’ulteriore iterazione

Si vuole così creare un dataframe contenente il titolo del film con la recensione corrispondente, che verrà salvato in un file csv.  

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd 

Si crea *lista* che conterrà il titolo del film, il link che porta alla recensione e la recensione completa.

In [ ]:
lista = []

for pagina in range(1,277): # PRIMA ITERAZIONE sulle pagine in modo da poter scaricare tutti i film disponibili
    URL_full = "https://www.rogerebert.com/reviews/page/" + str(pagina)
    page2 = requests.get(URL_full)
    soup2 = BeautifulSoup(page2.content, "html.parser")  # sorgente dell'i-esima pagina
    
    tags = soup2.find_all("h5", class_="review-stack--title") # per ogni pagina, viene estratto il tag contenente il titolo dei film presenti 
    # e il corrispondente link che porta alla recensione  
    lista_links=[]
    lista_titoli=[]
    for tag in tags:
        link = tag.a['href']  # estrazione link 
        titolo = tag.a.text   # estrazione titolo 
        lista_links.append(link) # lista con i links
        lista_titoli.append(titolo) # lista con i titoli 
    
    i=0 # SECONDA ITERAZIONE per le pagine con i link trovati nel punto precedente, in modo da ottenere le recensioni
    for link in lista_links:  #28 film
        URL = "https://www.rogerebert.com" + link 
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
      
        paragraphs = soup.find_all('section', class_="page-content--block_editor-content js--reframe") # estrazione sezione contenente la recensione completa
        recensione = ''
        for p in paragraphs:
            whole = p.get_text().strip() # estrazione paragrafi che compongono la recensione completa
            recensione = recensione + whole # unione dei paragrafi
            
        link = lista_links[i]
        titolo = lista_titoli[i]
        i=i+1

        print(titolo)
        lista.append([titolo, link, recensione]) # a ogni iterazione la lista finale viene aggiornata
  

Per ogni pagina vengono estratti 28 film, di cui gli ultimi 4 si ripetono, a ogni iterazione. Questi ultimi infatti, vengono evidenziati sulla pagina web come "popular reviews". Questo problema verrà risolto di seguito. <br>
Il **numero totale di film ottenuti ammonta a 7756**.

## **Storage** 

Si crea il dataset *recensioni.csv* e si salva in locale. Il file è vuoto e successivamente verrà riempito con i dati ottenuti tramite scraping. <br>
Le variabili sono *Title*, *Link* e *Review*.

In [ ]:
header = ['Title', 'Link', 'Review']
with open("recensioni.csv", 'w') as f: # creazione file "recensioni.csv" 
    csv_writer = csv.writer(f)
    csv_writer.writerow(header) # prima riga contenente l'header

Si salva *lista* in *recensioni.csv*.

In [ ]:
with open('recensioni.csv','a', encoding="utf-8") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerows(lista) # tutte le altre righe contengono gli elementi della lista creata nel punto precedente

Si importa il dataset e si chiama *recensioni*.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/MyDrive/Progetto Data Management/SCRAPING/recensioni.csv"

In [ ]:
#importazione recensioni.csv
import pandas as pd
recensioni=pd.read_csv(path)  

Si eliminano i duplicati. Si mantiene solo la prima recensione tra quelle che sono duplicate.  

In [ ]:
reviews = recensioni.drop_duplicates(subset="Title", keep='first') #
reviews = reviews[['Title', 'Review']] #creazione dataframe con le sole colonne di interesse 
reviews

,Title,Review
0,Jurassic World: Dominion,"Twenty-nine years ago, when ""Jurassic Park"" wa..."
1,Relative,Chicago-based writer/director Michael Glover S...
2,Keep Sweet: Pray and Obey,When does a true crime documentary cross the l...
3,The Janes,"Summer is superhero season, so it seems only f..."
4,Hustle,Adam Sandler is arguably the most basketball a...
...,...,...
7719,Extraordinary Measures,"""Extraordinary Measures"" is an ordinary film w..."
7720,Gigante,"We are all voyeurs, although some people fondl..."
7721,Tooth Fairy,In the pantheon of such legends as Santa Claus...
7722,A Town Called Panic,You know how kids play with little plastic act...


I film che si ripetono sono "Top Gun: Maverick, Interceptor, RRR, Eiffel". Attraverso la rimozione dei duplicati appena effettuata questi sono presenti solo una volta nel dataset, i duplicati sono eliminati. Il numero di film presenti adesso ammonta a 6525.

## **Data Cleaning e Data Quality: Completezza**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
reviews_finale = pd.read_csv("/content/drive/MyDrive/Progetto Data Management/SCRAPING/reviews.csv", encoding = 'utf-8')
reviews_finale

,Title,Review
0,Jurassic World: Dominion,"Twenty-nine years ago, when ""Jurassic Park"" wa..."
1,Relative,Chicago-based writer/director Michael Glover S...
2,Keep Sweet: Pray and Obey,When does a true crime documentary cross the l...
3,The Janes,"Summer is superhero season, so it seems only f..."
4,Hustle,Adam Sandler is arguably the most basketball a...
...,...,...
6520,Extraordinary Measures,"""Extraordinary Measures"" is an ordinary film w..."
6521,Gigante,"We are all voyeurs, although some people fondl..."
6522,Tooth Fairy,In the pantheon of such legends as Santa Claus...
6523,A Town Called Panic,You know how kids play with little plastic act...


In [ ]:
reviews_finale['Title'].isnull().sum()  # non ci sono dati mancanti in Title

0

In [ ]:
reviews_finale['Review'].isnull().sum()  # non ci sono dati mancanti in Review

0

In [ ]:
# data cleaning sul testo delle recensioni
i=0
while i<=6524:
  reviews_finale.loc[i]['Review'] = reviews_finale.loc[i]['Review'].replace('\xa0', ' ').replace('\n', '')  
  i+=1